In [3]:
import os

import logging
import mysql.connector
import pandas as pd
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime


logger = logging.getLogger()

In [4]:
years=range(1960,2023)
offsets=range(0,300,50)

In [283]:
spotify.search(q=f'year:2020', type='track', limit=50)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=year%3A2020&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4yvcSjfu4PC0CYQyLy4wSq'},
       'href': 'https://api.spotify.com/v1/artists/4yvcSjfu4PC0CYQyLy4wSq',
       'id': '4yvcSjfu4PC0CYQyLy4wSq',
       'name': 'Glass Animals',
       'type': 'artist',
       'uri': 'spotify:artist:4yvcSjfu4PC0CYQyLy4wSq'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
  

In [5]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

tracks_data_list = []

start = datetime.now()
print("start = ", start)
for year in years:
    for offset in offsets:
        try:
            track_results = spotify.search(q=f'year:{year}', type='track', limit=50, offset=offset)
            for track in track_results['tracks']['items']:
                single_track_dict = {                                       
                    'track_name': track['name'],
                    'artist_name': track['artists'][0]['name'],
                    'track_id': track['id'] ,
                    'popularity': track['popularity']
                }
                tracks_data_list.append(single_track_dict)
        except Exception as e:
            print(e)
            logger.info(e)
            continue
    print(f"Progression of years: {year}/{years[-1]}")
end = datetime.now()
print("finish = ", end)
print("duration = ", (end-start).total_seconds())

start =  2022-02-17 10:03:09.968972
Progression of years: 1960/2022
Progression of years: 1961/2022
Progression of years: 1962/2022
Progression of years: 1963/2022
Progression of years: 1964/2022
Progression of years: 1965/2022
Progression of years: 1966/2022
Progression of years: 1967/2022
Progression of years: 1968/2022
Progression of years: 1969/2022
Progression of years: 1970/2022
Progression of years: 1971/2022
Progression of years: 1972/2022
Progression of years: 1973/2022
Progression of years: 1974/2022
Progression of years: 1975/2022
Progression of years: 1976/2022
Progression of years: 1977/2022
Progression of years: 1978/2022
Progression of years: 1979/2022
Progression of years: 1980/2022
Progression of years: 1981/2022
Progression of years: 1982/2022
Progression of years: 1983/2022
Progression of years: 1984/2022
Progression of years: 1985/2022
Progression of years: 1986/2022
Progression of years: 1987/2022
Progression of years: 1988/2022
Progression of years: 1989/2022
Prog

In [243]:
len(tracks_data_list)

18900

In [244]:
df = pd.DataFrame(tracks_data_list)

Removing duplicates

In [245]:
df.duplicated().sum()

690

In [246]:
df.drop_duplicates(inplace=True)

In [268]:
df.shape

(18210, 4)

In [37]:
def extract_audio_feature(ids_list):
    list_features = []
    try:
        list_features = spotify.audio_features(ids_list)
    except Exception as e:
        print(e)
        logger.info(e)
    return list_features

In [38]:
tracks_features_list = []

total_tracks = df.shape[0]

start = datetime.now()
print("start = ", start)
for tracks_nbr in range(0, total_tracks, 100):
    print(f"Progression of tracks: {tracks_nbr}/{total_tracks}")
    tracks_features_list += extract_audio_feature(','.join(df.track_id[tracks_nbr:tracks_nbr+100]))
remaining_tracks = total_tracks%100
if remaining_tracks:
    tracks_features_list += extract_audio_feature(','.join(df.track_id[-remaining_tracks:]))
end = datetime.now()
print("finish = ", end)
print("duration = ", (end-start).total_seconds())

start =  2022-02-17 10:19:30.995732
Progression of tracks: 100/18210
Progression of tracks: 200/18210
Progression of tracks: 300/18210
Progression of tracks: 400/18210
Progression of tracks: 500/18210
Progression of tracks: 600/18210
Progression of tracks: 700/18210
Progression of tracks: 800/18210
Progression of tracks: 900/18210
Progression of tracks: 1000/18210
Progression of tracks: 1100/18210
Progression of tracks: 1200/18210
Progression of tracks: 1300/18210
Progression of tracks: 1400/18210
Progression of tracks: 1500/18210
Progression of tracks: 1600/18210
Progression of tracks: 1700/18210
Progression of tracks: 1800/18210
Progression of tracks: 1900/18210
Progression of tracks: 2000/18210
Progression of tracks: 2100/18210
Progression of tracks: 2200/18210
Progression of tracks: 2300/18210
Progression of tracks: 2400/18210
Progression of tracks: 2500/18210
Progression of tracks: 2600/18210
Progression of tracks: 2700/18210
Progression of tracks: 2800/18210
Progression of tracks

For some songs, the audio feature is not done so we can remove this songs

In [247]:
len(tracks_features_list)

18219

In [248]:
tracks_features_list = [track for track in tracks_features_list if isinstance(track, dict)]

In [249]:
len(tracks_features_list)

18219

In [269]:
df2 = pd.DataFrame(tracks_features_list)

Need to clean the duplicate again, it seems that the API requests are doubled sometimes. 

In [270]:
df2.shape

(18219, 18)

In [271]:
df2.duplicated().sum()
df2.drop_duplicates(inplace=True)
df2.shape

(18209, 18)

In [274]:
final_df = df.set_index('track_id').join(df2.set_index('id'), how='inner')

In [275]:
final_df.shape

(18209, 20)

In [276]:
final_df.sample(10)

,track_name,artist_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
19MkwMrRjO7773zpuGss1o,Shake the Frost (Live),Tyler Childers,72,0.447,0.341,2,-9.769,1,0.0457,0.80600,0.000002,0.6860,0.371,138.969,audio_features,spotify:track:19MkwMrRjO7773zpuGss1o,https://api.spotify.com/v1/tracks/19MkwMrRjO77...,https://api.spotify.com/v1/audio-analysis/19Mk...,175120,4
5xGsNXXTu545MWoeuFfjxT,It's Only Love,Bryan Adams,60,0.515,0.805,5,-4.491,1,0.0345,0.00993,0.000016,0.4190,0.743,101.161,audio_features,spotify:track:5xGsNXXTu545MWoeuFfjxT,https://api.spotify.com/v1/tracks/5xGsNXXTu545...,https://api.spotify.com/v1/audio-analysis/5xGs...,194618,4
4LGKzrdEeBwR9UHCGlDT0W,I'll Stick Around,Foo Fighters,61,0.306,0.985,0,-6.687,1,0.0822,0.00254,0.626000,0.1300,0.255,133.963,audio_features,spotify:track:4LGKzrdEeBwR9UHCGlDT0W,https://api.spotify.com/v1/tracks/4LGKzrdEeBwR...,https://api.spotify.com/v1/audio-analysis/4LGK...,232533,4
3rzenzs3lpi1XljKsZQ2Bl,Humpty Dumpty - 2006 Remastered Version,Marc Moulin,39,0.663,0.530,10,-9.199,0,0.0275,0.61300,0.906000,0.1090,0.631,103.779,audio_features,spotify:track:3rzenzs3lpi1XljKsZQ2Bl,https://api.spotify.com/v1/tracks/3rzenzs3lpi1...,https://api.spotify.com/v1/audio-analysis/3rze...,152400,4
0I0WdxFMMlF4tMwKmJzWdd,(Don't Go Back To) Rockville - Remastered 2006,R.E.M.,21,0.480,0.785,4,-7.765,1,0.0294,0.12000,0.000005,0.1380,0.921,151.752,audio_features,spotify:track:0I0WdxFMMlF4tMwKmJzWdd,https://api.spotify.com/v1/tracks/0I0WdxFMMlF4...,https://api.spotify.com/v1/audio-analysis/0I0W...,273080,4
6aL6pPBaTMGgM558mplSnj,I Can't Get Next To You,The Temptations,56,0.693,0.687,5,-12.909,1,0.0973,0.57500,0.000000,0.3980,0.699,108.854,audio_features,spotify:track:6aL6pPBaTMGgM558mplSnj,https://api.spotify.com/v1/tracks/6aL6pPBaTMGg...,https://api.spotify.com/v1/audio-analysis/6aL6...,174933,4
4dDurgTN3eGyU0ghzFJ0T1,Have a Drink on Me,AC/DC,59,0.421,0.889,2,-5.166,1,0.0545,0.00167,0.008380,0.6540,0.424,122.424,audio_features,spotify:track:4dDurgTN3eGyU0ghzFJ0T1,https://api.spotify.com/v1/tracks/4dDurgTN3eGy...,https://api.spotify.com/v1/audio-analysis/4dDu...,238467,4
1g2eRmWpv2pfVDOgsYKiBS,One Love,Whodini,45,0.853,0.542,4,-15.431,0,0.1460,0.04080,0.000003,0.0473,0.715,91.561,audio_features,spotify:track:1g2eRmWpv2pfVDOgsYKiBS,https://api.spotify.com/v1/tracks/1g2eRmWpv2pf...,https://api.spotify.com/v1/audio-analysis/1g2e...,333000,4
1oht5GevPN9t1T3kG1m1GO,Fire and Rain - 2019 Remaster,James Taylor,74,0.597,0.271,5,-17.293,1,0.0394,0.76600,0.011900,0.0933,0.338,76.271,audio_features,spotify:track:1oht5GevPN9t1T3kG1m1GO,https://api.spotify.com/v1/tracks/1oht5GevPN9t...,https://api.spotify.com/v1/audio-analysis/1oht...,203613,4
0bYwN4LqZYLfs9ZEHPXjBC,Best Thing That Ever Happened to Me,Gladys Knight & The Pips,59,0.352,0.545,1,-11.939,0,0.0871,0.30900,0.056300,0.1060,0.459,189.498,audio_features,spotify:track:0bYwN4LqZYLfs9ZEHPXjBC,https://api.spotify.com/v1/tracks/0bYwN4LqZYLf...,https://api.spotify.com/v1/audio-analysis/0bYw...,224987,4


In [277]:
final_df.describe()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,18209.000000,1.820900e+04,18209.000000
mean,53.534681,0.570275,0.594110,5.239167,-9.506448,0.705420,0.071309,0.314549,0.097785,0.198077,0.560166,120.248642,2.349242e+05,3.913065
std,19.853799,0.166637,0.237698,3.521069,4.475365,0.455866,0.079428,0.312929,0.244145,0.173292,0.251719,28.945378,9.038462e+04,0.371238
min,0.000000,0.000000,0.000020,0.000000,-47.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.062200e+04,0.000000
25%,47.000000,0.458000,0.423000,2.000000,-11.930000,0.000000,0.033000,0.031200,0.000000,0.091800,0.362000,97.867000,1.811330e+05,4.000000
50%,57.000000,0.578000,0.612000,5.000000,-8.755000,1.000000,0.042200,0.201000,0.000049,0.129000,0.569000,119.397000,2.230790e+05,4.000000
75%,67.000000,0.689000,0.791000,8.000000,-6.275000,1.000000,0.067500,0.568000,0.009100,0.252000,0.771000,136.837000,2.703330e+05,4.000000
max,100.000000,0.988000,1.000000,11.000000,0.899000,1.000000,0.954000,0.996000,1.000000,0.997000,0.993000,243.372000,1.644773e+06,5.000000


In [278]:
final_df.dtypes

track_name           object
artist_name          object
popularity            int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
dtype: object

key is a reserved keyword in mysql, either change the name or surround it with backticks \`key\`

In [282]:
start = datetime.now()
print("start = ", start)
try:
    mydb = mysql.connector.connect(
        host=os.environ['MYSQL_SPOTIFY_HOST'],
        user=os.environ['MYSQL_SPOTIFY_USER'],
        password=os.environ['MYSQL_SPOTIFY_PW']
    )

    mycursor = mydb.cursor()

    mycursor.execute("CREATE DATABASE IF NOT EXISTS wilts_songs;")

    mydb = mysql.connector.connect(
        host=os.environ['MYSQL_SPOTIFY_HOST'],
        user=os.environ['MYSQL_SPOTIFY_USER'],
        password=os.environ['MYSQL_SPOTIFY_PW'],
        database="wilts_songs"
    )

    mycursor = mydb.cursor()
    mycursor.execute("""DROP TABLE IF EXISTS songs""")
    mycursor.execute("""CREATE TABLE IF NOT EXISTS songs (
    id VARCHAR(55) NOT NULL PRIMARY KEY,
    track_name VARCHAR(250),
    artist_name VARCHAR(100),
    popularity TINYINT,
    duration_ms INT,
    danceability FLOAT,
    time_signature TINYINT,
    energy FLOAT,
    song_key TINYINT,
    loudness FLOAT,
    mode TINYINT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT
    ) ENGINE=InnoDB;""")

    mysql_insert_query = """INSERT INTO songs (id, track_name, artist_name, popularity, duration_ms,
        danceability, time_signature, energy, song_key, loudness, mode, speechiness, acousticness,
        instrumentalness, liveness, valence, tempo)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

    selected_cols = ['track_name', 'artist_name', 'popularity', 'duration_ms',
        'danceability', 'time_signature', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo']

    records = [record.item() for record in final_df[selected_cols].to_records()]
    mycursor.executemany(mysql_insert_query, records)

    mydb.commit()
except mysql.connector.Error as error:
    print("Failed to insert into MySQL table {}".format(error))

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")
    end = datetime.now()
    print("finish = ", end)
    print("duration = ", (end-start).total_seconds())
#  EVENTUELLEMENT AJOUTER DECONNEXION DB ET CURSEUR ET TRY ETC
#     try:
#         connection = mysql.connector.connect(host='localhost',
#                                              database='Electronics',
#                                              user='pynative',
#                                              password='pynative@#29')
#         cursor = connection.cursor()
#         mySql_insert_query = """INSERT INTO Laptop (Id, Name, Price, Purchase_date) 
#                                 VALUES (%s, %s, %s, %s) """

#         record = (id, name, price, purchase_date)
#         cursor.execute(mySql_insert_query, record)
#         connection.commit()
#         print("Record inserted successfully into Laptop table")

#     except mysql.connector.Error as error:
#         print("Failed to insert into MySQL table {}".format(error))

#     finally:
#         if connection.is_connected():
#             cursor.close()
#             connection.close()
#             print("MySQL connection is closed")


start =  2022-02-17 15:04:43.986991
MySQL connection is closed
finish =  2022-02-17 15:04:45.136979
duration =  1.149988


In [241]:
final_df.loc['07zVQBJfbOuaAhpT3stRFL']

,track_name,artist_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
07zVQBJfbOuaAhpT3stRFL,Stay Soft,Mitski,75,0.602,0.678,7,-8.656,1,0.0299,0.0404,0.00075,0.121,0.552,104.969,audio_features,spotify:track:07zVQBJfbOuaAhpT3stRFL,https://api.spotify.com/v1/tracks/07zVQBJfbOua...,https://api.spotify.com/v1/audio-analysis/07zV...,196433,4
07zVQBJfbOuaAhpT3stRFL,Stay Soft,Mitski,75,0.602,0.678,7,-8.656,1,0.0299,0.0404,0.00075,0.121,0.552,104.969,audio_features,spotify:track:07zVQBJfbOuaAhpT3stRFL,https://api.spotify.com/v1/tracks/07zVQBJfbOua...,https://api.spotify.com/v1/audio-analysis/07zV...,196433,4


In [233]:
final_df.track_name.apply(lambda x: len(x)).describe()

count    18219.000000
mean        21.607937
std         14.282020
min          2.000000
25%         12.000000
50%         18.000000
75%         28.000000
max        164.000000
Name: track_name, dtype: float64

In [229]:
for record in records[:100]:
    print(record[1])
    print(len(record[1]))

African Ghost - Pitch -4
24
Swing, Swing
12
Mr. Brightside
14
All Cried Out (with Full Force)
31
Historia de Taxi
16
I Kissed A Girl
15
Unconditionally
15
One Right Now (with The Weeknd)
31
Lieder ohne Worte (Songs without Words), Book 2, Op. 30: No. 7 in E-Flat Major, Op. 30, No. 1
93
Two Lovers
10
I'm Coming On - 2017 Remaster
29
Sands of Time - Single; 2018 Remaster
37
Bajo La Luna
12
IV. Sweatpants
14
I Love You So Much It Hurts
27
The Boy In the Bubble
21
Sk8er Boi
9
Sleigh Ride
11
Get The Funk Out Ma Face
24
Who Wants To Live Forever - Remastered 2011
43
Ain't No Future In Yo' Frontin'
31
Stay (I Missed You)
19
As Long as You Love Me
22
The Weight
10
Unchained Melody
16
Pa' Mayte
9
Already Christmas (Prod. by Yang Da Il)
39
Strawberry Letter 23
20
The Big Kahuna - Ashes to Ashes B-Side; 2016 Remaster
53
Big Girls Don't Cry
19
Favorite Day of the Year
24
Tomorrow Never Knows - Remastered 2009
38
K-Love Fan Awards: Songs of the Year (2014 Mash-Up)
51
Papa's Got A Brand New Bag
26
P

In [214]:
for record in records[:3]:
#     record_temp = record.astype(dtype_new)
    item = record.item()
    print(item)
    print(len(item))
    print(mysql_insert_query % (item))

('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d', 0, 61362, 0.598, 4, 0.644, 6, -10.913, 0, 0.0865, 0.253, 0.194, 0.162, 0.375, 192.127)
17
INSERT INTO songs (id, track_name, artist_name, popularity, duration_ms,
    danceability, time_signature, energy, song_key, loudness, mode, speechiness, acousticness,
    instrumentalness, liveness, valence, tempo)
    VALUES ('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d', 0, 61362, 0.598000, 4, 0.644000, 6, -10.913000, 0, 0.086500, 0.253000, 0.194000, 0.162000, 0.375000, 192.127000)
('003FTlCpBTM4eSqYSWPv4H', 'Swing, Swing', 'The All-American Rejects', 70, 233267, 0.553, 4, 0.717, 7, -5.857, 1, 0.0318, 6.48e-05, 0.128, 0.0385, 0.318, 127.947)
17
INSERT INTO songs (id, track_name, artist_name, popularity, duration_ms,
    danceability, time_signature, energy, song_key, loudness, mode, speechiness, acousticness,
    instrumentalness, liveness, valence, tempo)
    VALUES ('003FTlCpBTM4eSqYSWPv4H', 'Swing, Sw

In [158]:
sept = set([1,2,3,4,5,5,2])

In [160]:
huit = set([1,2,5,8])

In [165]:
tracks_features_list

[{'danceability': 0.274,
  'energy': 0.348,
  'key': 5,
  'loudness': -8.631,
  'mode': 1,
  'speechiness': 0.0293,
  'acousticness': 0.547,
  'instrumentalness': 0.0133,
  'liveness': 0.334,
  'valence': 0.328,
  'tempo': 87.43,
  'type': 'audio_features',
  'id': '4Hhv2vrOTy89HFRcjU3QOx',
  'uri': 'spotify:track:4Hhv2vrOTy89HFRcjU3QOx',
  'track_href': 'https://api.spotify.com/v1/tracks/4Hhv2vrOTy89HFRcjU3QOx',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Hhv2vrOTy89HFRcjU3QOx',
  'duration_ms': 179693,
  'time_signature': 3},
 {'danceability': 0.421,
  'energy': 0.285,
  'key': 2,
  'loudness': -9.43,
  'mode': 0,
  'speechiness': 0.0293,
  'acousticness': 0.829,
  'instrumentalness': 1.55e-06,
  'liveness': 0.233,
  'valence': 0.402,
  'tempo': 85.861,
  'type': 'audio_features',
  'id': '0zGLlXbHlrAyBN1x6sY0rb',
  'uri': 'spotify:track:0zGLlXbHlrAyBN1x6sY0rb',
  'track_href': 'https://api.spotify.com/v1/tracks/0zGLlXbHlrAyBN1x6sY0rb',
  'analysis_url': 'https://ap

In [163]:
set(tracks_features_list)

TypeError: unhashable type: 'dict'

In [189]:
records = final_df[selected_cols].to_records()

In [190]:
records

rec.array([('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d',  0,  61362, 0.598, 4, 0.644 ,  6, -10.913, 0, 0.0865, 2.53e-01, 0.194 , 0.162 , 0.375, 192.127),
           ('003FTlCpBTM4eSqYSWPv4H', 'Swing, Swing', 'The All-American Rejects', 70, 233267, 0.553, 4, 0.717 ,  7,  -5.857, 1, 0.0318, 6.48e-05, 0.128 , 0.0385, 0.318, 127.947),
           ('003vvx7Niy0yvhvHt4a68B', 'Mr. Brightside', 'The Killers', 85, 222973, 0.352, 4, 0.911 ,  1,  -5.23 , 1, 0.0747, 1.21e-03, 0.    , 0.0995, 0.236, 148.033),
           ...,
           ('7zto61V8ySp03Qi6X1LU2X', 'Tin Pan Alley (AKA Roughest Place in Town)', 'Stevie Ray Vaughan', 56, 551787, 0.576, 3, 0.0968,  6, -19.439, 0, 0.036 , 5.46e-01, 0.286 , 0.226 , 0.168, 113.232),
           ('7zvzNcr3ZbTEaIBzwybD30', 'One Love (Mixed)', 'Maor Levi', 27, 103040, 0.455, 4, 0.929 , 10,  -7.907, 0, 0.0651, 4.65e-06, 0.368 , 0.574 , 0.234, 128.266),
           ('7zwn1eykZtZ5LODrf7c0tS', 'You Get Me So High', 'The Neighbourhood', 84, 153

In [200]:
for record in records[:3]:
#     record_temp = record.astype(dtype_new)
    print(record_temp.item())
    for item in record_temp.item():
        print(type(item))
        print(item)


('003vvx7Niy0yvhvHt4a68B', 'Mr. Brightside', 'The Killers', 85, 222973, 0.352, 4, 0.911, 1, -5.23, 1, 0.0747, 0.00121, 0.0, 0.0995, 0.236, 148.033)
<class 'str'>
003vvx7Niy0yvhvHt4a68B
<class 'str'>
Mr. Brightside
<class 'str'>
The Killers
<class 'int'>
85
<class 'int'>
222973
<class 'float'>
0.352
<class 'int'>
4
<class 'float'>
0.911
<class 'int'>
1
<class 'float'>
-5.23
<class 'int'>
1
<class 'float'>
0.0747
<class 'float'>
0.00121
<class 'float'>
0.0
<class 'float'>
0.0995
<class 'float'>
0.236
<class 'float'>
148.033
('003vvx7Niy0yvhvHt4a68B', 'Mr. Brightside', 'The Killers', 85, 222973, 0.352, 4, 0.911, 1, -5.23, 1, 0.0747, 0.00121, 0.0, 0.0995, 0.236, 148.033)
<class 'str'>
003vvx7Niy0yvhvHt4a68B
<class 'str'>
Mr. Brightside
<class 'str'>
The Killers
<class 'int'>
85
<class 'int'>
222973
<class 'float'>
0.352
<class 'int'>
4
<class 'float'>
0.911
<class 'int'>
1
<class 'float'>
-5.23
<class 'int'>
1
<class 'float'>
0.0747
<class 'float'>
0.00121
<class 'float'>
0.0
<class 'float

In [170]:
import numpy as np

In [194]:
names = ['id', 'track_name', 'artist_name', 'popularity', 'duration_ms',
    'danceability', 'time_signature', 'energy', 'song_key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo']
formats = [object, object, object, int,
           int, float, int, float,
           int, float, int, float, float,
           float, float, float, float]

In [195]:
dtype_new = np.dtype({'names': names, 'formats': formats})

In [186]:
records2 = records.copy().astype(dtype_new)

In [185]:
records

rec.array([('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d',  0,  61362, 0.598, 4, 0.644 ,  6, -10.913, 0, 0.0865, 2.53e-01, 0.194 , 0.162 , 0.375, 192.127),
           ('003FTlCpBTM4eSqYSWPv4H', 'Swing, Swing', 'The All-American Rejects', 70, 233267, 0.553, 4, 0.717 ,  7,  -5.857, 1, 0.0318, 6.48e-05, 0.128 , 0.0385, 0.318, 127.947),
           ('003vvx7Niy0yvhvHt4a68B', 'Mr. Brightside', 'The Killers', 85, 222973, 0.352, 4, 0.911 ,  1,  -5.23 , 1, 0.0747, 1.21e-03, 0.    , 0.0995, 0.236, 148.033),
           ...,
           ('7zto61V8ySp03Qi6X1LU2X', 'Tin Pan Alley (AKA Roughest Place in Town)', 'Stevie Ray Vaughan', 56, 551787, 0.576, 3, 0.0968,  6, -19.439, 0, 0.036 , 5.46e-01, 0.286 , 0.226 , 0.168, 113.232),
           ('7zvzNcr3ZbTEaIBzwybD30', 'One Love (Mixed)', 'Maor Levi', 27, 103040, 0.455, 4, 0.929 , 10,  -7.907, 0, 0.0651, 4.65e-06, 0.368 , 0.574 , 0.234, 128.266),
           ('7zwn1eykZtZ5LODrf7c0tS', 'You Get Me So High', 'The Neighbourhood', 84, 153

In [178]:
records.dtype

dtype((numpy.record, [('index', 'O'), ('track_name', 'O'), ('artist_name', 'O'), ('popularity', '<i8'), ('duration_ms', '<i8'), ('danceability', '<f8'), ('time_signature', '<i8'), ('energy', '<f8'), ('key', '<i8'), ('loudness', '<f8'), ('mode', '<i8'), ('speechiness', '<f8'), ('acousticness', '<f8'), ('instrumentalness', '<f8'), ('liveness', '<f8'), ('valence', '<f8'), ('tempo', '<f8')]))

In [187]:
records2.dtype

dtype((numpy.record, [('id', '<U'), ('track_name', '<U'), ('artist_name', '<U'), ('popularity', '<i8'), ('duration_ms', '<i8'), ('danceability', '<f8'), ('time_signature', '<i8'), ('energy', '<f8'), ('song_key', '<i8'), ('loudness', '<f8'), ('mode', '<i8'), ('speechiness', '<f8'), ('acousticness', '<f8'), ('instrumentalness', '<f8'), ('liveness', '<f8'), ('valence', '<f8'), ('tempo', '<f8')]))

In [188]:
for record in records2:
    print(record.item())

('', '', '', 0, 61362, 0.598, 4, 0.644, 6, -10.913, 0, 0.0865, 0.253, 0.194, 0.162, 0.375, 192.127)
('', '', '', 70, 233267, 0.553, 4, 0.717, 7, -5.857, 1, 0.0318, 6.48e-05, 0.128, 0.0385, 0.318, 127.947)
('', '', '', 85, 222973, 0.352, 4, 0.911, 1, -5.23, 1, 0.0747, 0.00121, 0.0, 0.0995, 0.236, 148.033)
('', '', '', 45, 284973, 0.48, 4, 0.435, 1, -9.959, 1, 0.0305, 0.207, 5.83e-06, 0.196, 0.328, 144.435)
('', '', '', 68, 405120, 0.715, 4, 0.621, 2, -10.755, 0, 0.0341, 0.117, 1.07e-06, 0.279, 0.698, 147.053)
('', '', '', 73, 179640, 0.699, 4, 0.76, 5, -3.173, 1, 0.0677, 0.00223, 0.0, 0.132, 0.696, 129.996)
('', '', '', 81, 228879, 0.432, 4, 0.725, 7, -4.862, 1, 0.0431, 0.00273, 0.0, 0.208, 0.353, 128.902)
('', '', '', 92, 193507, 0.687, 4, 0.781, 1, -4.806, 1, 0.053, 0.0361, 0.0, 0.0755, 0.688, 97.014)
('', '', '', 62, 282813, 0.355, 4, 0.0164, 3, -28.659, 1, 0.0396, 0.995, 0.926, 0.0986, 0.0922, 67.946)
('', '', '', 51, 166733, 0.678, 4, 0.405, 2, -16.965, 1, 0.0304, 0.426, 0.0, 0.109

('', '', '', 54, 543560, 0.759, 4, 0.675, 3, -14.366, 0, 0.0422, 0.0261, 0.0341, 0.431, 0.866, 127.522)
('', '', '', 72, 241067, 0.636, 4, 0.405, 8, -11.113, 1, 0.0287, 0.355, 0.0, 0.0804, 0.257, 128.068)
('', '', '', 56, 365760, 0.904, 4, 0.624, 3, -5.936, 0, 0.045, 0.00622, 0.00171, 0.0304, 0.804, 115.735)
('', '', '', 16, 216093, 0.661, 4, 0.183, 2, -12.854, 0, 0.029, 0.894, 0.00104, 0.0859, 0.508, 94.911)
('', '', '', 53, 243507, 0.523, 4, 0.854, 9, -8.27, 1, 0.0954, 0.478, 0.000939, 0.271, 0.83, 146.688)
('', '', '', 51, 250867, 0.715, 4, 0.827, 1, -8.555, 1, 0.0369, 0.14, 0.176, 0.243, 0.901, 116.213)
('', '', '', 0, 226413, 0.718, 4, 0.711, 8, -5.746, 0, 0.11, 0.157, 5.52e-05, 0.1, 0.92, 173.916)
('', '', '', 48, 256533, 0.57, 4, 0.0463, 4, -21.521, 0, 0.0448, 0.945, 0.79, 0.112, 0.377, 139.262)
('', '', '', 70, 309173, 0.708, 4, 0.774, 11, -6.931, 0, 0.0372, 0.149, 0.00109, 0.169, 0.903, 126.636)
('', '', '', 60, 173733, 0.539, 3, 0.114, 2, -20.575, 1, 0.0346, 0.882, 0.00902, 0

('', '', '', 4, 88760, 0.589, 4, 0.936, 10, -8.429, 0, 0.0417, 0.00512, 0.864, 0.128, 0.144, 128.789)
('', '', '', 74, 153205, 0.392, 4, 0.629, 5, -7.056, 1, 0.0335, 0.718, 3.76e-06, 0.129, 0.72, 124.511)
('', '', '', 78, 237787, 0.546, 4, 0.529, 11, -10.511, 0, 0.0583, 0.0862, 6.5e-06, 0.124, 0.247, 107.331)
('', '', '', 8, 197600, 0.65, 4, 0.663, 9, -10.053, 0, 0.0971, 0.49, 0.000165, 0.219, 0.543, 138.016)
('', '', '', 44, 251067, 0.946, 4, 0.469, 5, -13.091, 0, 0.0865, 0.0608, 0.00126, 0.257, 0.866, 110.969)
('', '', '', 69, 289133, 0.744, 4, 0.766, 0, -6.375, 1, 0.0265, 0.0873, 0.0, 0.509, 0.61, 127.966)
('', '', '', 60, 290493, 0.581, 4, 0.886, 0, -6.505, 1, 0.0324, 0.279, 0.0297, 0.691, 0.586, 106.067)
('', '', '', 15, 298241, 0.656, 4, 0.995, 1, -5.988, 1, 0.0418, 0.00133, 0.703, 0.135, 0.701, 124.998)
('', '', '', 55, 314440, 0.531, 4, 0.668, 8, -9.494, 1, 0.115, 0.218, 0.00404, 0.307, 0.434, 134.446)
('', '', '', 64, 200690, 0.414, 3, 0.514, 8, -8.4, 1, 0.0405, 0.676, 4.19e-0

('', '', '', 57, 256667, 0.716, 4, 0.758, 0, -9.395, 1, 0.0271, 0.0569, 0.00515, 0.353, 0.857, 120.774)
('', '', '', 7, 43627, 0.425, 4, 0.926, 0, -8.376, 1, 0.042, 2.63e-06, 0.904, 0.435, 0.474, 132.032)
('', '', '', 52, 193760, 0.146, 3, 0.355, 7, -14.583, 1, 0.0352, 0.865, 0.835, 0.114, 0.178, 146.408)
('', '', '', 59, 180520, 0.673, 4, 0.785, 3, -7.041, 1, 0.0407, 0.114, 0.000904, 0.119, 0.562, 119.78)
('', '', '', 55, 309867, 0.802, 4, 0.594, 7, -9.72, 1, 0.268, 0.0782, 0.0, 0.282, 0.548, 89.659)
('', '', '', 55, 220040, 0.493, 4, 0.306, 0, -17.392, 1, 0.029, 0.678, 0.243, 0.14, 0.346, 71.985)
('', '', '', 59, 318947, 0.563, 4, 0.421, 7, -12.567, 1, 0.0311, 0.822, 2.28e-05, 0.122, 0.463, 139.374)
('', '', '', 57, 305400, 0.444, 4, 0.454, 5, -7.434, 1, 0.0419, 0.892, 0.0546, 0.114, 0.236, 67.848)
('', '', '', 65, 232588, 0.93, 4, 0.607, 7, -10.389, 1, 0.0457, 0.0526, 5.22e-06, 0.039, 0.95, 118.242)
('', '', '', 73, 238560, 0.456, 4, 0.573, 5, -3.866, 0, 0.0275, 0.578, 3.05e-06, 0.1

('', '', '', 77, 193893, 0.611, 4, 0.95, 8, -3.509, 1, 0.0518, 0.00141, 1.59e-06, 0.0543, 0.607, 132.078)
('', '', '', 55, 349280, 0.734, 4, 0.859, 7, -7.237, 0, 0.0922, 0.215, 0.343, 0.0823, 0.908, 112.941)
('', '', '', 57, 291400, 0.791, 4, 0.186, 10, -11.367, 1, 0.0333, 0.625, 2.67e-06, 0.0574, 0.317, 114.061)
('', '', '', 43, 171587, 0.292, 3, 0.116, 9, -20.635, 1, 0.0377, 0.991, 0.89, 0.108, 0.232, 67.324)
('', '', '', 69, 159000, 0.79, 4, 0.41, 0, -13.241, 1, 0.0343, 0.263, 0.00314, 0.174, 0.815, 112.345)
('', '', '', 80, 271093, 0.332, 4, 0.214, 4, -12.518, 1, 0.0349, 0.845, 5.62e-06, 0.0839, 0.11, 67.531)
('', '', '', 39, 219053, 0.761, 4, 0.924, 2, -3.781, 0, 0.0331, 0.0578, 0.0, 0.0541, 0.78, 126.01)
('', '', '', 58, 371773, 0.506, 4, 0.624, 9, -4.997, 0, 0.0362, 0.646, 0.0, 0.104, 0.664, 94.979)
('', '', '', 12, 247302, 0.422, 4, 0.815, 11, -5.698, 0, 0.0406, 0.0695, 2.94e-06, 0.354, 0.433, 121.921)
('', '', '', 54, 101160, 0.589, 4, 0.522, 2, -8.718, 1, 0.0521, 0.53, 0.0002

('', '', '', 52, 231400, 0.551, 4, 0.969, 6, -2.41, 1, 0.0373, 0.0727, 7.55e-05, 0.0991, 0.431, 136.507)
('', '', '', 50, 162400, 0.634, 4, 0.612, 1, -12.881, 1, 0.263, 0.479, 0.0, 0.159, 0.691, 78.817)
('', '', '', 54, 204133, 0.873, 4, 0.62, 7, -8.812, 1, 0.293, 0.0113, 0.000121, 0.0925, 0.574, 105.332)
('', '', '', 43, 362520, 0.306, 4, 0.88, 9, -5.535, 1, 0.0485, 0.0128, 0.653, 0.0964, 0.617, 106.964)
('', '', '', 49, 349933, 0.62, 4, 0.42, 5, -15.481, 0, 0.0343, 0.0129, 0.81, 0.0587, 0.945, 97.107)
('', '', '', 56, 196107, 0.164, 4, 0.974, 9, -8.399, 1, 0.0921, 0.00922, 0.000359, 0.238, 0.209, 177.821)
('', '', '', 69, 239680, 0.371, 4, 0.758, 7, -5.679, 1, 0.036, 0.0958, 0.0, 0.116, 0.452, 77.81)
('', '', '', 13, 314000, 0.259, 3, 0.506, 4, -8.697, 1, 0.0294, 0.155, 0.0254, 0.0833, 0.669, 179.065)
('', '', '', 38, 144627, 0.206, 4, 0.99, 10, -3.565, 1, 0.13, 0.00176, 8.07e-05, 0.335, 0.666, 187.433)
('', '', '', 32, 272417, 0.509, 4, 0.482, 1, -11.643, 1, 0.0388, 0.44, 0.000258, 

('', '', '', 70, 462093, 0.336, 4, 0.787, 10, -8.803, 0, 0.0732, 0.0793, 0.0648, 0.32, 0.383, 140.003)
('', '', '', 3, 253920, 0.82, 4, 0.452, 1, -9.796, 1, 0.0348, 0.543, 0.00294, 0.0714, 0.737, 117.401)
('', '', '', 45, 221573, 0.629, 4, 0.33, 7, -13.713, 1, 0.0308, 0.531, 2.79e-05, 0.0801, 0.501, 144.122)
('', '', '', 61, 327040, 0.325, 3, 0.856, 4, -6.547, 0, 0.188, 0.000319, 0.00708, 0.257, 0.652, 69.955)
('', '', '', 62, 169293, 0.661, 4, 0.571, 8, -12.317, 1, 0.103, 0.27, 0.0, 0.98, 0.826, 122.396)
('', '', '', 69, 280636, 0.567, 4, 0.559, 9, -10.175, 1, 0.0288, 0.00888, 0.00131, 0.0764, 0.898, 127.227)
('', '', '', 39, 161707, 0.437, 3, 0.295, 0, -10.648, 1, 0.026, 0.764, 0.000399, 0.113, 0.269, 105.617)
('', '', '', 72, 173040, 0.521, 4, 0.487, 6, -7.525, 0, 0.0299, 0.253, 0.0, 0.0579, 0.483, 117.072)
('', '', '', 51, 203187, 0.34, 3, 0.427, 1, -10.042, 1, 0.0312, 0.11, 0.615, 0.122, 0.404, 120.646)
('', '', '', 56, 197640, 0.667, 4, 0.776, 11, -7.292, 0, 0.0356, 0.272, 3.59e-

In [157]:
for record in final_df[selected_cols].to_records()[:3]:
    for i in range(len(record)):
        record = record.item()
        print(record)
        if type(record[i]) == "<class 'numpy.int64'>":
            print("INTEUR")
        if not isinstance(record[i], str):
            print(type(record[i].item()))

('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d', 0, 61362, 0.598, 4, 0.644, 6, -10.913, 0, 0.0865, 0.253, 0.194, 0.162, 0.375, 192.127)


AttributeError: 'tuple' object has no attribute 'item'

In [87]:
selected_cols = ['track_name', 'artist_name', 'popularity', 'duration_ms',
    'danceability', 'time_signature', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo']
final_df[selected_cols].to_records()[0]

('002Ni4R1NeoWLjfO7SoL2T', 'African Ghost - Pitch -4', 'Bong trip d', 0, 61362, 0.598, 4, 0.644, 6, -10.913, 0, 0.0865, 0.253, 0.194, 0.162, 0.375, 192.127)

In [212]:
if 12:
    print("ok")

ok


In [208]:
spotify.audio_features(','.join(df.track_id[0:3]))

[{'danceability': 0.274,
  'energy': 0.348,
  'key': 5,
  'loudness': -8.631,
  'mode': 1,
  'speechiness': 0.0293,
  'acousticness': 0.547,
  'instrumentalness': 0.0133,
  'liveness': 0.334,
  'valence': 0.328,
  'tempo': 87.43,
  'type': 'audio_features',
  'id': '4Hhv2vrOTy89HFRcjU3QOx',
  'uri': 'spotify:track:4Hhv2vrOTy89HFRcjU3QOx',
  'track_href': 'https://api.spotify.com/v1/tracks/4Hhv2vrOTy89HFRcjU3QOx',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Hhv2vrOTy89HFRcjU3QOx',
  'duration_ms': 179693,
  'time_signature': 3},
 {'danceability': 0.421,
  'energy': 0.285,
  'key': 2,
  'loudness': -9.43,
  'mode': 0,
  'speechiness': 0.0293,
  'acousticness': 0.829,
  'instrumentalness': 1.55e-06,
  'liveness': 0.233,
  'valence': 0.402,
  'tempo': 85.861,
  'type': 'audio_features',
  'id': '0zGLlXbHlrAyBN1x6sY0rb',
  'uri': 'spotify:track:0zGLlXbHlrAyBN1x6sY0rb',
  'track_href': 'https://api.spotify.com/v1/tracks/0zGLlXbHlrAyBN1x6sY0rb',
  'analysis_url': 'https://ap

In [178]:
from itertools import chain


In [191]:
for i in range(0,11,2):
    print(i)

0
2
4
6
8
10


In [206]:
x=1333
interval = chain(range(0,x,100), [x]) if x%100 != 0 else range(0,x,100)
for i in interval:
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1333


In [187]:
for i in chain(range(0,1333,100), [1333]):
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1333


In [169]:
features = spotify.audio_features('04rdnfHk3O3HGJB32geDlM')[0]

In [174]:
features

{'danceability': 0.462,
 'energy': 0.387,
 'key': 0,
 'loudness': -7.401,
 'mode': 1,
 'speechiness': 0.0291,
 'acousticness': 0.568,
 'instrumentalness': 0,
 'liveness': 0.164,
 'valence': 0.0428,
 'tempo': 122.711,
 'type': 'audio_features',
 'id': '04rdnfHk3O3HGJB32geDlM',
 'uri': 'spotify:track:04rdnfHk3O3HGJB32geDlM',
 'track_href': 'https://api.spotify.com/v1/tracks/04rdnfHk3O3HGJB32geDlM',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/04rdnfHk3O3HGJB32geDlM',
 'duration_ms': 261080,
 'time_signature': 4}

In [161]:
df_features = pd.DataFrame()

In [167]:
df_features.append({'danceability': 0.462,
 'energy': 0.387,
 'key': 0,
 'loudness': -7.401,
 'mode': 1,
 'speechiness': 0.0291,
 'acousticness': 0.568,
 'instrumentalness': 0,
 'liveness': 0.164,
 'valence': 0.0428,
 'tempo': 122.711,
 'type': 'audio_features',
 'id': '04rdnfHk3O3HGJB32geDlM',
 'uri': 'spotify:track:04rdnfHk3O3HGJB32geDlM',
 'track_href': 'https://api.spotify.com/v1/tracks/04rdnfHk3O3HGJB32geDlM',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/04rdnfHk3O3HGJB32geDlM',
 'duration_ms': 261080,
 'time_signature': 4}, ignore_index=True)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.568,https://api.spotify.com/v1/audio-analysis/04rd...,0.462,261080.0,0.387,04rdnfHk3O3HGJB32geDlM,0.0,0.0,0.164,-7.401,1.0,0.0291,122.711,4.0,https://api.spotify.com/v1/tracks/04rdnfHk3O3H...,audio_features,spotify:track:04rdnfHk3O3HGJB32geDlM,0.0428


In [168]:
df_features

""


In [140]:
range(0,160,50)

range(0, 160, 50)

In [141]:
160%50

10

TypeError: unsupported operand type(s) for +: 'range' and 'int'

In [3]:
def get_tracks_data(spotify_client, query, chunk_size, max_results):
    tracks_data_list = []

    for i in range(0, max_results, chunk_size):
        try:
            track_results = spotify_client.search(q=query, type='track', limit=50,offset=i)
            for track in track_results['tracks']['items']:
                print(track)
                for key,values in track.items():
                    single_track_dict = {                                       
                        'track_name': track['name'],
                        'artist_name': track['artists'][0]['name'],
                        'track_id': track['id'] ,
                        'popularity': track['popularity']
                    }
                    tracks_data_list.append(single_track_dict)
        except Exception as e:
            print(e)
            logger.info(e)
            continue
        return tracks_data_list

In [4]:
client_id = os.environ['SPOTIPY_CLIENT_ID']
client_secret = os.environ['SPOTIPY_CLIENT_SECRET']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

tracks_data_list = get_tracks_data(spotify_client, 'year:2021', 50, 1000)

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2021', 'limit': 50, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


http status: 404, code:-1 - https://api.spotify.com/v1/search?q=year%3A2021&limit=50&offset=0&type=track:
 Not found., reason: None


In [5]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])



Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [25]:
pp = pprint.PrettyPrinter(indent=2)

In [91]:
year=2021
spotify.search(q=f'year:{year}', type='track', limit=50)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=year%3A2021&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4aXXDj9aZnlshx7mzj3W1N'},
       'href': 'https://api.spotify.com/v1/artists/4aXXDj9aZnlshx7mzj3W1N',
       'id': '4aXXDj9aZnlshx7mzj3W1N',
       'name': 'Lin-Manuel Miranda',
       'type': 'artist',
       'uri': 'spotify:artist:4aXXDj9aZnlshx7mzj3W1N'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/5NTEjLv3GEX1Vx0ITx9Pxe'},
       'href': 'https://api.spotify.com/v1/artists/5NTEjLv3GEX1Vx0ITx9Pxe',
       'id': '5NTEjLv3GEX1Vx0ITx9Pxe',
       'name': 'Germaine Franco',
       'type': 'artist',
       'uri': 'spotify:artist:5NTEjLv3GEX1Vx0ITx9Pxe'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/3xLU748QxpTmIVaiNXXg0P'},
       'href': 'https://api.spotify.com/v1/artists/3xLU748QxpTmIVaiNXXg0P',
       'id': '3

In [95]:
for key, values in spotify.search(q='year:2021', type='track', limit=2, offset=522)['tracks'].items():
    print("CLEF",key)
    print(values)
    print('\n')

CLEF href
https://api.spotify.com/v1/search?query=year%3A2021&type=track&offset=522&limit=2


CLEF items
[{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5'}, 'href': 'https://api.spotify.com/v1/artists/6l3HvQ5sa6mXTsMTB19rO5', 'id': '6l3HvQ5sa6mXTsMTB19rO5', 'name': 'J. Cole', 'type': 'artist', 'uri': 'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB

In [125]:
json = spotify.search(q='year:2021', type='track', limit=2, offset=522)

In [129]:
for track in json['tracks']['items']:
    print('track_name: ', track['name'])
    print('artist_name: ', track['artists'][0]['name'])
    print('track_id: ', track['id'])
    print('popularity: ', track['popularity'])

track_name:  a m a r i
artist_name:  J. Cole
track_id:  2cnKST6T9qUo2i907lm8zX
popularity:  78
track_name:  Blame (Year in Review 2021)
artist_name:  Cosmic Gate
track_id:  573Mn78Ofpo0BtYD5XmYrK
popularity:  19


In [126]:
for track in json['tracks']['items']:
    print(track)
    for key,values in track.items():
        print("CLEF",key)
        print(values)
        print('\n')

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5'}, 'href': 'https://api.spotify.com/v1/artists/6l3HvQ5sa6mXTsMTB19rO5', 'id': '6l3HvQ5sa6mXTsMTB19rO5', 'name': 'J. Cole', 'type': 'artist', 'uri': 'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', '

In [101]:
print(json['tracks']['items'][0]['artists'][0]['id'])
print(json['tracks']['items'][0]['artists'][0]['name'])
print(json['tracks']['items'][0]['popularity'])
print(json['tracks']['items'][0]['id'])
print(json['tracks']['items'][0]['name'])

# 0: dépend de limit (nombre de tracks prises par requête)
# Autre 0: dérouler pour avoir tous les artistes dans une liste


6l3HvQ5sa6mXTsMTB19rO5
J. Cole
78
2cnKST6T9qUo2i907lm8zX
a m a r i


In [24]:
import pprint

In [ ]:
for year in range()